## 제품명 검색 결과 url 추출

In [32]:
import pandas as pd

df = pd.read_excel("의약품.xlsx")
df.head()

,disease,medicine
0,소화불량,트라부틴정
1,소화불량,훼스탈 골드
2,소화불량,훼스탈플러스정
3,편두통,미가펜 에스
4,편두통,마이드린


In [33]:
from bs4 import BeautifulSoup
import requests   
import re
from tqdm.notebook import tqdm

links = []

for i in tqdm(range(len(df['medicine']))):
    try:
        url = "https://terms.naver.com/medicineSearch.nhn?mode=nameSearch&query={}".format(df['medicine'][i])
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        title = soup.select('li div strong.title')[0]

        for a in title.find_all('a', href = re.compile("^(/entry.nhn)")):
                link = a.attrs['href']
                links.append("https://terms.naver.com/" + link)
    except:
        links.append('x')

In [34]:
urls = pd.DataFrame(data = {'links' : links }) ; urls

,links
0,https://terms.naver.com//entry.nhn?docId=21239...
1,https://terms.naver.com//entry.nhn?docId=21353...
2,https://terms.naver.com//entry.nhn?docId=21353...
3,https://terms.naver.com//entry.nhn?docId=58662...
4,https://terms.naver.com//entry.nhn?docId=21399...
...,...
135,https://terms.naver.com//entry.nhn?docId=21290...
136,https://terms.naver.com//entry.nhn?docId=21417...
137,https://terms.naver.com//entry.nhn?docId=21243...
138,https://terms.naver.com//entry.nhn?docId=21290...


In [35]:
urls.nunique()

links    103
dtype: int64

In [55]:
data = pd.concat([df, urls], axis = 1)
data.head()

,disease,medicine,Unnamed: 0,links
0,소화불량,트라부틴정,0,https://terms.naver.com//entry.nhn?docId=21239...
1,소화불량,훼스탈 골드,1,https://terms.naver.com//entry.nhn?docId=21353...
2,소화불량,훼스탈플러스정,2,https://terms.naver.com//entry.nhn?docId=21353...
3,편두통,미가펜 에스,3,https://terms.naver.com//entry.nhn?docId=58662...
4,편두통,마이드린,4,https://terms.naver.com//entry.nhn?docId=21399...


In [121]:
data.to_excel('의약품.xlsx')

## 제품 url 불러와 상세정보 크롤링

In [95]:
from bs4 import BeautifulSoup
import requests   

제품 = []
제조사 = []
성분정보 = []
저장방법 = []
효능효과 = []
용법용량 = []
주의사항 = []
이미지 = []

url = data['links'][0]
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
제품.append(soup.select('h2.headword')[0].text)
제조사.append(soup.select('tbody td')[2].text)
성분정보.append(soup.select('p.txt')[1].text)
저장방법.append(soup.select('p.txt')[2].text)
효능효과.append(soup.select('p.txt')[3].text)
용법용량.append(soup.select('p.txt')[4].text)
주의사항.append(soup.select('p.txt')[5].text)
이미지.append(set(img['data-src'] for img in soup.select('div.thmb.c > a img') if img.has_attr('data-src')))

print(제품)
print(제조사[0].strip('\n'))
print(성분정보[0].strip('\n'))
print(저장방법[0].strip('\n'))
print(효능효과[0].strip('\n'))
print(용법용량[0].strip('\n'))
print(주의사항[0].strip('\n'))
print(이미지)

['트라부틴정']
                                        경남제약

                                
트리메부틴말레산염 100mg
밀폐용기,실온(1~30℃)보관
1. 주효능 효과1) 식도역류 및 열공헤르니아, 위, 십이지장염, 위, 십이지장궤양에 있어서의 소화기능이상 (복통(배아픔), 소화불량, 구역, 구토)2) 과민성대장증후군 및 경련성 결장3) 소아 질환 : 습관성 구토, 비감염성 장관통과장애 (변비, 설사), 동요(안절부절)자극, 수면장애
성인 : 트리메부틴말레산염으로서 1회 100-200mg 1일 3회 식전에 경구투여한다(복용한다).연령(나이), 증상에 따라 적절히 증감한다.
1. 다음 환자에는 투여하지 말 것이 약은 유당(젖당)수화물을 함유하고 있으므로, 갈락토오스 불내성(galactose intolerance), Lapp 유당(젖당)분해효소 결핍증(Lapp lactase deficiency) 또는 포도당-갈락토오스 흡수장애(glucose-galactose malabsorption) 등의 유전적인 문제가 있는 환자에게는 투여하면 안 된다.2. 부작용1) 소화기계 : 드물게 변비, 설사, 복명(창자 가스소리), 구역, 구토, 소화장애, 구갈(목마름), 구내(입안)마비감 등이 나타날 수 있다.2) 순환기계 : 드물게 심계항진(두근거림)이 나타날 수 있다.3) 정신신경계 : 드물게 피로감, 졸음, 현기(어지러움), 권태감, 두통 등이 나타날수 있다.4) 간장 : 드물게 GOT, GPT 상승이 나타날 수 있다.5) 과민증 : 드물게 발진 등이 나타날 수 있으므로 이러한 증상이 나타나는 경우에는 투여를 중지한다.3. 임부에 대한 투여임신중의 투여에 대한 안전성은 확립되어 있지 않으므로 임부 또는 임신하고 있을 가능성이 있는 부인에는 치료상의 유익성이 위험성을 상회(웃돎)한다고 판단되는 경우에만 투여한다.4. 수유부에 대한 투여수유중의 투여에 대한 안전성은 확립되어 있지 않으므로 수유중인 부인에는 투여를 

In [91]:
from tqdm.notebook import tqdm
import time
import re

result_list = []

for link in tqdm((data['links'])):
    if link != 'x':
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'html.parser')

        try:
            제품 = soup.select('h2.headword')[0].text
            제조사 = re.sub("[\r\n\t]", "", str(soup.select('tbody td')[2].text.strip()))
            성분정보 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[1].text.strip()))
            저장방법 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[2].text.strip()))
            효능효과 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[3].text.strip()))
            용법용량 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[4].text.strip()))
            주의사항 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[5].text.strip()))
            이미지 = (set(img['data-src'] for img in soup.select('div.thmb.c > a img') if img.has_attr('data-src')))
        except:
            제품 = 'x'
            제조사 = 'x'
            성분정보 = 'x'
            저장방법 = 'x'
            효능효과 = 'x'
            용법용량 = 'x'
            주의사항 = 'x'
            이미지 = 'x'

        result = 제품, 제조사, 성분정보, 저장방법, 효능효과, 용법용량, 주의사항, 이미지
        result_list.append(result)

        time.sleep(0.5)

In [92]:
medicine_data = pd.DataFrame(result_list, columns = ['제품', '제조사', '성분정보', '저장방법', '효능효과', '용법용량', '주의사항', '이미지'])

In [93]:
medicine_data.head()

,제품,제조사,성분정보,저장방법,효능효과,용법용량,주의사항,이미지
0,트라부틴정,경남제약,트리메부틴말레산염 100mg,"밀폐용기,실온(1~30℃)보관","1. 주효능 효과1) 식도역류 및 열공헤르니아, 위, 십이지장염, 위, 십이지장궤양...",성인 : 트리메부틴말레산염으로서 1회 100-200mg 1일 3회 식전에 경구투여한...,"1. 다음 환자에는 투여하지 말 것이 약은 유당(젖당)수화물을 함유하고 있으므로, ...",{https://dthumb-phinf.pstatic.net/?src=%22http...
1,훼스탈골드정,한독,"시메티콘 60mg, 우르소데옥시콜산 20mg, 판크레아틴 150mg, 프로자임6 1...","실온보관(30℃이하), 밀폐용기","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","성인 1회 1정, 1일 3회 식사 후 씹지 말고 복용한다.",1. 다음과 같은 사람은 이 약을 복용하지 말 것.1) 만 7세 이하의 어린이2. ...,{https://dthumb-phinf.pstatic.net/?src=%22http...
2,훼스탈플러스정,한독,"시메티콘 30mg, 우르소데옥시콜산 10mg, 판크레아틴 315mg, 셀룰라제 10mg","밀폐용기, 실온보관","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","성인 1회 1-2정, 소아(만 8세~만15세미만) 1회 1정1일 3회, 식사 후 씹...",{https://dthumb-phinf.pstatic.net/?src=%22http...
3,미가펜에스연질캡슐,우리들제약,나프록센 250mg,"기밀용기, 실온(1∼30℃)보관","1. 주효능효과류마티양 관절염, 골관절염(퇴행성 관절질환), 강직성 척추염, 건염,...","1. 류마티양 관절염, 골관절염, 강직성 척추염성인 : 나프록센으로서 1회 250～...",1. 경고1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열진...,{}
4,마이드린캡슐,녹십자,"디클로랄페나존 100mg, 아세트아미노펜 325mg, 이소메텝텐뮤케이트 65mg","기밀용기,실온보관(1~30℃)",긴장성 두통 및 혈관성 두통 (편두통),1.긴장성 두통:성인 1회 1~2캅셀을 1일 3회까지 복용한다.복용간격은 4시간 이...,"[허가사항 변경지시(안전성정보 처리) (의약품관리총괄과-11696호, 2014.12...",{https://dthumb-phinf.pstatic.net/?src=%22http...


In [107]:
list(medicine_data["이미지"][0])

['https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3323_000_9%2F20180711200502152_A4FO7DVH6.png%2FP_2009090700205_.png%3Ftype%3Dm4500_4500_fst%26wm%3DN%22&twidth=520&theight=287&opts=17']

In [108]:
def name_cut(x):
    x = list(x)
    return x 

In [110]:
medicine_data["이미지"] = medicine_data["이미지"].apply(name_cut)

In [111]:
medicine_data.head()

,제품,제조사,성분정보,저장방법,효능효과,용법용량,주의사항,이미지
0,트라부틴정,경남제약,트리메부틴말레산염 100mg,"밀폐용기,실온(1~30℃)보관","1. 주효능 효과1) 식도역류 및 열공헤르니아, 위, 십이지장염, 위, 십이지장궤양...",성인 : 트리메부틴말레산염으로서 1회 100-200mg 1일 3회 식전에 경구투여한...,"1. 다음 환자에는 투여하지 말 것이 약은 유당(젖당)수화물을 함유하고 있으므로, ...",[https://dthumb-phinf.pstatic.net/?src=%22http...
1,훼스탈골드정,한독,"시메티콘 60mg, 우르소데옥시콜산 20mg, 판크레아틴 150mg, 프로자임6 1...","실온보관(30℃이하), 밀폐용기","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","성인 1회 1정, 1일 3회 식사 후 씹지 말고 복용한다.",1. 다음과 같은 사람은 이 약을 복용하지 말 것.1) 만 7세 이하의 어린이2. ...,[https://dthumb-phinf.pstatic.net/?src=%22http...
2,훼스탈플러스정,한독,"시메티콘 30mg, 우르소데옥시콜산 10mg, 판크레아틴 315mg, 셀룰라제 10mg","밀폐용기, 실온보관","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","성인 1회 1-2정, 소아(만 8세~만15세미만) 1회 1정1일 3회, 식사 후 씹...",[https://dthumb-phinf.pstatic.net/?src=%22http...
3,미가펜에스연질캡슐,우리들제약,나프록센 250mg,"기밀용기, 실온(1∼30℃)보관","1. 주효능효과류마티양 관절염, 골관절염(퇴행성 관절질환), 강직성 척추염, 건염,...","1. 류마티양 관절염, 골관절염, 강직성 척추염성인 : 나프록센으로서 1회 250～...",1. 경고1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열진...,[]
4,마이드린캡슐,녹십자,"디클로랄페나존 100mg, 아세트아미노펜 325mg, 이소메텝텐뮤케이트 65mg","기밀용기,실온보관(1~30℃)",긴장성 두통 및 혈관성 두통 (편두통),1.긴장성 두통:성인 1회 1~2캅셀을 1일 3회까지 복용한다.복용간격은 4시간 이...,"[허가사항 변경지시(안전성정보 처리) (의약품관리총괄과-11696호, 2014.12...",[https://dthumb-phinf.pstatic.net/?src=%22http...


In [113]:
(''.join(medicine_data["이미지"][0]))

'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3323_000_9%2F20180711200502152_A4FO7DVH6.png%2FP_2009090700205_.png%3Ftype%3Dm4500_4500_fst%26wm%3DN%22&twidth=520&theight=287&opts=17'

In [114]:
def name_cut(x):
    x = (''.join(x))
    return x 

In [115]:
medicine_data["이미지"] = medicine_data["이미지"].apply(name_cut)

In [116]:
medicine_data.head()

,제품,제조사,성분정보,저장방법,효능효과,용법용량,주의사항,이미지
0,트라부틴정,경남제약,트리메부틴말레산염 100mg,"밀폐용기,실온(1~30℃)보관","1. 주효능 효과1) 식도역류 및 열공헤르니아, 위, 십이지장염, 위, 십이지장궤양...",성인 : 트리메부틴말레산염으로서 1회 100-200mg 1일 3회 식전에 경구투여한...,"1. 다음 환자에는 투여하지 말 것이 약은 유당(젖당)수화물을 함유하고 있으므로, ...",https://dthumb-phinf.pstatic.net/?src=%22https...
1,훼스탈골드정,한독,"시메티콘 60mg, 우르소데옥시콜산 20mg, 판크레아틴 150mg, 프로자임6 1...","실온보관(30℃이하), 밀폐용기","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","성인 1회 1정, 1일 3회 식사 후 씹지 말고 복용한다.",1. 다음과 같은 사람은 이 약을 복용하지 말 것.1) 만 7세 이하의 어린이2. ...,https://dthumb-phinf.pstatic.net/?src=%22https...
2,훼스탈플러스정,한독,"시메티콘 30mg, 우르소데옥시콜산 10mg, 판크레아틴 315mg, 셀룰라제 10mg","밀폐용기, 실온보관","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","소화불량, 식욕감퇴(식욕부진), 과식, 식체(위체(체함)), 소화촉진, 소화불량으로...","성인 1회 1-2정, 소아(만 8세~만15세미만) 1회 1정1일 3회, 식사 후 씹...",https://dthumb-phinf.pstatic.net/?src=%22https...
3,미가펜에스연질캡슐,우리들제약,나프록센 250mg,"기밀용기, 실온(1∼30℃)보관","1. 주효능효과류마티양 관절염, 골관절염(퇴행성 관절질환), 강직성 척추염, 건염,...","1. 류마티양 관절염, 골관절염, 강직성 척추염성인 : 나프록센으로서 1회 250～...",1. 경고1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열진...,
4,마이드린캡슐,녹십자,"디클로랄페나존 100mg, 아세트아미노펜 325mg, 이소메텝텐뮤케이트 65mg","기밀용기,실온보관(1~30℃)",긴장성 두통 및 혈관성 두통 (편두통),1.긴장성 두통:성인 1회 1~2캅셀을 1일 3회까지 복용한다.복용간격은 4시간 이...,"[허가사항 변경지시(안전성정보 처리) (의약품관리총괄과-11696호, 2014.12...",https://dthumb-phinf.pstatic.net/?src=%22https...


In [118]:
medicine_data.to_excel("의약품 정보.xlsx")